#### Author : Maximiliano Lopez Salgado
#### First commit: 2023-05-10
#### Last commit: 2023-05-10
#### Description: This notebook is used to perform EDA on the bike data set

In [ ]:
# import ML libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import folium
from folium.plugins import HeatMap

<center><h1>Exploratory Data Analysis</center></h1>

In [ ]:
#### Formulate some hypothesis and plot the data to check if the hypothesis are true. 
#### I.E: Hourly trend: most of the businesses have rush hours and weak hours. It should not be an exception for bike rentals as well. 
#### Daily Trend: weekdays vs weekends and registered users vs casual users.. 
#### Rain: the demand for bikes might be changed on a rainy day as compared to a sunny day. Similarly, people prefer to go out on less humid days..
#### Temperature: in warm countries, temperature generally keeps people inside. You have to check Washington DC's temperature for making any guess.. 
#### Business model: businesses often rely on registered customers more than casual users. There might be some interesting insights that can strengthen this assumption.

In [ ]:
# import bike data
bike = pd.read_csv('../data_wrangling/bike_cleaned.csv')

# print first 5 rows
bike.head()

In [ ]:
### Hipotesis 1

In [ ]:
# Hipotesis 1: if the value of the the season column is equal to summer, then the number of bike rentals will be higher than the other seasons
# create a pivot table to check the number of bike rentals in each season and order it in descending order
bike_season = bike.pivot_table(index='season', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_season

In [ ]:
#### Hipotesis is true, the number of bike rentals is higher during the summer season

In [ ]:
### Hipotesis 2

In [ ]:
# Hipotesis 2: if the value of the the weather column is equal to clean, then the number of bike rentals will be higher than the other seasons
# Create a pivot table to check the number of bike rentals in each weather and order it in descending order
bike_weather = pd.pivot_table(bike, index='weather', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_weather

In [ ]:
#### Hipotesis is true, the number of bike rentals is higher when the weather is clear

In [ ]:
### Hipotesis 3

In [ ]:
# Hipotesis 3: if the value of the the temperature column is equal to hot, then the number of bike rentals will be higher than the other seasons
# Create a pivot table to check the number of bike rentals in each warmness and order it in descending orde
bike_warmness = pd.pivot_table(bike, index='warmness', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_warmness

In [ ]:
#### Hipotesis 3 is not true, the number of bike rentals is higher when the temperature is very hot

In [ ]:
### Hipotesis 4

In [ ]:
# hipotesis 4: if the value of the the working day column is equal to yes, then the number of bike rentals by registered members will be higher than the non registered members and vice versa
# use the columns casual and registered to create a pivot table to check the number of bike rentals for each type of customer on workingdays and order it in descending order
bike_users = pd.pivot_table(bike, index='workingday', values=['casual_user', 'registered_user'], aggfunc='sum')
bike_users

In [ ]:
#### Hipotesis 4 is true, registered customers rent more bikes than casual customers either in working days or not. Also casual users rent more bikes during weekdays

In [ ]:
### Hipotesis 5

In [ ]:
# Hipotesis 5: if the value of the the day period column is equal to night, then the number of bike rentals will be higher than the other periods
# Create a pivot table to check the number of bike rentals in each day period
bike_day_period = pd.pivot_table(bike, index='day_period', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_day_period

In [ ]:
#### Hipotesis 5 is false, the number of bike rentals is higher in the afternoon

In [ ]:
### Hipotesis 6

In [ ]:
# Hipotesis 6: if the value of the the weekday column is equal to monday, tuesday, wednesday, thursday and/or friday, then the number of bike rentals will be higher than the other days
# Create a pivot table to check the number of bike rentals in each weekday
bike_weekday = pd.pivot_table(bike, index='weekday', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_weekday

In [ ]:
#### Hipotesis 6 is not true, the number of bike rentals is higher in friday and thursday, but on saturday the number of bike rentals is bigger than the number of bike rentals in wednesday, tuesday and monday

In [ ]:
### Hipotesis 7

In [ ]:
# Hipotesis 7: if the value of the the month column is equal to june, july and/or august, then the number of bike rentals will be higher than the other months
# Create a pivot table to check the number of bike rentals in each month
bike_month = pd.pivot_table(bike, index='month', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_month

In [ ]:
#### Hipotesis 7 is not true, because despite that the number of bike rentals in august and june are the top 2 on the records, the 3rd month with the highest number of bike rentals is september, which is not a summer month

In [ ]:
### Hipotesis 8

In [ ]:
# Hipotesis 8: if the value of the the hour column is equal to 8, 17 and/or 18, then the number of bike rentals will be higher than the other hours
# Create a pivot table to check the number of bike rentals in each hour
bike_hour = pd.pivot_table(bike, index='hour', values='count', aggfunc='sum').sort_values(by='count', ascending=False)
bike_hour

In [ ]:
#### Hipotesis 8 is not true, because the number of bike rentals is higher in the hour 17 and 18, but not in the hour 8